# Test Mirror Sequence Data Generator

This notebook tests the mirror sequence data generation functions from `func/data_mirror_seq.py`.

In [1]:
import torch
from data_mirror_seq import make_seq, masking, make_record, make_batch
from types import SimpleNamespace

In [2]:
# Configuration
config = SimpleNamespace(
    n_input_values=10,  # Range of possible values (1 to n_input_values-1, 0 reserved for mask)
    seq_len=8,         # Length of the sequence
    mask_frac=0.3,     # Fraction of positions to mask
    batch_size=4       # Number of sequences in a batch
)

## Test Single Sequence Generation

In [3]:
# Test make_seq
seq = make_seq(config.n_input_values, config.seq_len)
print("Generated sequence:")
print(seq)

# Verify the mirror property
half_len = config.seq_len // 2
first_half = seq[:, :half_len]
second_half = seq[:, half_len:]
print("\nFirst half:", first_half)
print("Second half:", second_half)
print("Second half (flipped):", second_half.flip(1))
print("\nIs mirrored correctly:", torch.equal(first_half, second_half.flip(1)))

Generated sequence:
tensor([[2, 9, 9, 7, 7, 9, 9, 2]])

First half: tensor([[2, 9, 9, 7]])
Second half: tensor([[7, 9, 9, 2]])
Second half (flipped): tensor([[2, 9, 9, 7]])

Is mirrored correctly: True


## Test Masking

In [4]:
# Test masking
seq = make_seq(config.n_input_values, config.seq_len)
masked_seq, mask = masking(seq, config.mask_frac)

print("Original sequence:")
print(seq)
print("\nMask (True indicates masked positions):")
print(mask)
print("\nMasked sequence (0 indicates masked positions):")
print(masked_seq)

# Calculate masking statistics
mask_ratio = mask.float().mean().item()
print(f"\nActual mask ratio: {mask_ratio:.2f} (target: {config.mask_frac})")

Original sequence:
tensor([[8, 2, 5, 1, 1, 5, 2, 8]])

Mask (True indicates masked positions):
tensor([[False,  True,  True,  True, False, False, False, False]])

Masked sequence (0 indicates masked positions):
tensor([[8, 0, 0, 0, 1, 5, 2, 8]])

Actual mask ratio: 0.38 (target: 0.3)


## Test Single Record Generation

In [5]:
# Test make_record
record = make_record(config)

print("Record contents:")
for key, value in record.items():
    print(f"\n{key}:")
    print(value)
    print(f"Shape: {value.shape}")

Record contents:

pos_id:
tensor([[0, 1, 2, 3, 4, 5, 6, 7]])
Shape: torch.Size([1, 8])

input:
tensor([[0, 1, 3, 8, 8, 3, 1, 6]])
Shape: torch.Size([1, 8])

target:
tensor([[6, 1, 3, 8, 8, 3, 1, 6]])
Shape: torch.Size([1, 8])

mask:
tensor([[ True, False, False, False, False, False, False, False]])
Shape: torch.Size([1, 8])


## Test Batch Generation

In [6]:
# Test make_batch
batch = make_batch(config)

print("Batch contents:")
for key, value in batch.items():
    print(f"\n{key}:")
    print(value)
    print(f"Shape: {value.shape}")

# Verify batch dimensions
expected_shape = (config.batch_size, config.seq_len)
print(f"\nAll tensors have expected shape {expected_shape}:", 
      all(v.shape == expected_shape for v in batch.values()))

Batch contents:

pos_id:
tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7]])
Shape: torch.Size([4, 8])

input:
tensor([[1, 2, 4, 0, 4, 4, 0, 1],
        [8, 0, 5, 0, 2, 5, 6, 8],
        [2, 0, 2, 0, 5, 2, 4, 2],
        [0, 0, 0, 2, 2, 6, 0, 0]])
Shape: torch.Size([4, 8])

target:
tensor([[1, 2, 4, 4, 4, 4, 2, 1],
        [8, 6, 5, 2, 2, 5, 6, 8],
        [2, 4, 2, 5, 5, 2, 4, 2],
        [4, 7, 6, 2, 2, 6, 7, 4]])
Shape: torch.Size([4, 8])

mask:
tensor([[False, False, False,  True, False, False,  True, False],
        [False,  True, False,  True, False, False, False, False],
        [False,  True, False,  True, False, False, False, False],
        [ True,  True,  True, False, False, False,  True,  True]])
Shape: torch.Size([4, 8])

All tensors have expected shape (4, 8): True
